<a href="https://colab.research.google.com/github/jhyoo78/jhyoo78/blob/main/skip_gram_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.manual_seed(10)
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from sklearn import decomposition
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10,8)

from nltk.corpus import stopwords
import nltk  # natural Language toolkit(python)
nltk.download('stopwords')
nltk.download('punkt')  # 추가 --> 없으면 3번째 노트북 페이지의 nltk.word_tokenize(i)에서 오류 발생
from nltk.corpus import stopwords  #Import stopwords

In [ ]:
corpus = [
    'drink milk',
    'drink cold water',
    'drink cold cola',
    'drink juice',
    'drink cola',
    'eat bacon',
    'eat mango',
    'eat cherry',
    'eat apple',
    'juice with sugar',
    'cola with sugar',
    'mango is fruit',
    'apple is fruit',
    'cherry is fruit',
    'Berlin is Germany',
    'Boston is USA',
    'Mercedes from Germany',
    'Mercedes is a car',
    'Ford from USA',
    'Ford is a car'
]

In [ ]:
def create_vocabulary(corpus):
    '''말 뭉치 내의 모든 단어(ID로 식별)에 대해 등장 순서대로 하나의 dictionary를 생성한다. 빈도순서가 아님'''
    vocabulary = {}
    i = 0
    for s in corpus:
        for w in s.split():   # s.split()은 문자열 s를 (구분자)에 따라 분리하여 list로 만듬
            if w not in vocabulary:
                vocabulary[w] = i
                #print(vocabulary)  # 추가된 dict 내용을 포함하여 전체를 출력
                i+=1
    return vocabulary

def prepare_set(corpus, n_gram = 1):
    '''이웃 단어들에 대해 입력 열과 출력 열로 구성된 dataset을 생성한다. Creates a dataset with Input column and Outputs columns for neighboring words.
       이 경우 이웃 관계의 수 = n_gram x 2 이다 '''
    columns = ['Input'] + [f'Output{i+1}' for i in range(n_gram*2)]
    result = pd.DataFrame(columns = columns)
    for sentence in corpus:
        for i,w in enumerate(sentence.split()):
            inp = [w]
            out = []
            for n in range(1,n_gram+1):
                # look back
                if (i-n)>=0:
                    out.append(sentence.split()[i-n])
                else:
                    out.append('<padding>')

                # look forward
                if (i+n)<len(sentence.split()):
                    out.append(sentence.split()[i+n])
                else:
                    out.append('<padding>')
            row = pd.DataFrame([inp+out], columns = columns)
            result = result.append(row, ignore_index = True)
    return result

def prepare_set_ravel(corpus, n_gram = 1):
    '''이웃 단어들에 대해 입력 열과 출력 열로 구성된 dataset을 생성한다.
          이 경우 이웃 관계의 수 = n_gram x 2 이다 '''
    columns = ['Input', 'Output']           # column 이름을 지정하여 df를 생성한다.
    result = pd.DataFrame(columns = columns)
    for sentence in corpus:
        for i,w in enumerate(sentence.split()):
            inp = w
            for n in range(1,n_gram+1):
                # look back
                if (i-n)>=0:
                    out = sentence.split()[i-n]
                    row = pd.DataFrame([[inp,out]], columns = columns)
                    result = result.append(row, ignore_index = True)

                # look forward
                if (i+n)<len(sentence.split()):
                    out = sentence.split()[i+n]
                    row = pd.DataFrame([[inp,out]], columns = columns)
                    result = result.append(row, ignore_index = True)
    return result

In [ ]:
# 불용어(the, a, is, from, with 등)를 제거한다.

stop_words = set(stopwords.words('english'))

def preprocess(corpus):
    result = []
    for i in corpus:
        out = nltk.word_tokenize(i)
        out = [x.lower() for x in out]
        out = [x for x in out if x not in stop_words]
        result.append(" ". join(out))
    return result

corpus = preprocess(corpus)
corpus

In [ ]:
# Dictionary를 만든다. 빈도 순서가 아니고 등장 순서로 index가 분여됨.

vocabulary = create_vocabulary(corpus)
vocabulary

In [ ]:
# 각 문장에 대해 단어의 조합을 만든다.

train_emb = prepare_set(corpus, n_gram = 2)
train_emb.head(10)

In [ ]:
# n-gram을 2로 하여 답어 조합을 만든다.

train_emb = prepare_set_ravel(corpus, n_gram = 2)
train_emb.head()

In [ ]:
# 앞에서 출력한 각 단어를 index로 바꾼다.

train_emb.Input = train_emb.Input.map(vocabulary)   # df에서 Input 열을 vocabulary의 index로 변환한다. 단, train_emb와 vocabulary에 동일한 열이 존재해야 한다.
print(train_emb.head() )
train_emb.Output = train_emb.Output.map(vocabulary) # df에서 Output 열을 vocabulary의 index로 변환한다.
train_emb.head()

In [ ]:
vocab_size = len(vocabulary)

def get_input_tensor(tensor):  # 단어 인덱스의 1차원 텐서를 one-hot 인코딩된 2D 텐서로 변환한다. 이 함수는 입력계층에서만 사용한다.
    size = [*tensor.shape][0]
    inp = torch.zeros(size, vocab_size).scatter_(1, tensor.unsqueeze(1), 1.)
    return Variable(inp).float()

In [ ]:
embedding_dims = 5
device = torch.device('cpu')

In [ ]:
# 단순 확인 용이므로 삭제할 것

a= torch.rand(3, 3)   # 0.0 ~ 1.0 사이의 값을 균등분포로 출력,
print(a)
print(torch.randn(3,3))  # 평균이 0 이고 분산이 1인 가우시안(표준) 정규 분포로 출력

In [ ]:
initrange = 0.5 / embedding_dims

'''무작위 값을 갖는 Tensor를 생성하고, Variable로 감싼다. requires_grade=True로 설정하여 역전파 중에 이 Variable들에 대한 기울기를 계산할 수 있게 한다'''
'''Variable()은 autograd(자동 미분)을 위해 사용되었으나 2018년 이후 torch.tensor로 통합되었음'''

W1 = Variable(torch.randn(vocab_size, embedding_dims, device=device).uniform_(-initrange, initrange).float(), requires_grad=True)  # shape V*H
# torch.rand는 uniform 분포로 균등하게 출력
# torch.randn은 Gausian normal distribution으로 출력, unifrom_(a, b)는 [a, b] 범위의 값으로 unifrom random matrix를 생성, device는 앞에서 정의한 cpu를 사용,
W2 = Variable(torch.randn(embedding_dims, vocab_size, device=device).uniform_(-initrange, initrange).float(), requires_grad=True)  # shape H*V
print(f'W1 shape is: {W1.shape}, W2 shape is: {W2.shape}')
print(W1)
print(W2)

In [ ]:
num_epochs = 2000
learning_rate = 2e-1   # 0.2
lr_decay = 0.99
loss_hist = []

In [ ]:
%%time
for epo in range(num_epochs):
    for x,y in zip(DataLoader(train_emb.Input.values, batch_size=train_emb.shape[0]), DataLoader(train_emb.Output.values, batch_size=train_emb.shape[0])):

        # one-hot encode input tensor
        input_tensor = get_input_tensor(x) #shape N*V

        # simple NN architecture
        h = input_tensor.mm(W1) # shape 1*H
        y_pred = h.mm(W2) # shape 1*V

        # define loss func
        loss_f = torch.nn.CrossEntropyLoss()     # see details: https://pytorch.org/docs/stable/nn.html

        #compute loss
        loss = loss_f(y_pred, y)

        # bakpropagation step
        loss.backward()

        # Update weights using gradient descent. For this step we just want to mutate
        # the values of w1 and w2 in-place; we don't want to build up a computational
        # graph for the update steps, so we use the torch.no_grad() context manager
        # to prevent PyTorch from building a computational graph for the updates
        with torch.no_grad():
            # SGD optimization is implemented in PyTorch, but it's very easy to implement manually providing better understanding of process
            W1 -= learning_rate*W1.grad.data  # 참고 --> https://9bow.github.io/PyTorch-tutorials-kr-0.3.1/beginner/pytorch_with_examples.html#pytorch-variables-autograd
            W2 -= learning_rate*W2.grad.data
            # zero gradients for next step
            W1.grad.data.zero_()
            W1.grad.data.zero_()
    if epo%10 == 0:
        learning_rate *= lr_decay
    loss_hist.append(loss)
    if epo%50 == 0:
        print(f'Epoch {epo}, loss = {loss}')


In [ ]:
W1 = W1.detach().numpy()
W2 = W2.T.detach().numpy()
print(W1)

In [ ]:
svd = decomposition.TruncatedSVD(n_components=2)
W1_dec = svd.fit_transform(W1)
x = W1_dec[:,0]
y = W1_dec[:,1]
plot = sns.scatterplot(x, y)

for i in range(0,W1_dec.shape[0]):
     plot.text(x[i], y[i]+2e-2, list(vocabulary.keys())[i], horizontalalignment='center', size='small', color='black', weight='semibold');

In [ ]:
W2_dec = svd.fit_transform(W2)
x1 = W2_dec[:,0]
y1 = W2_dec[:,1]
plot1 = sns.scatterplot(x1, y1)
for i in range(0,W2_dec.shape[0]):
     plot1.text(x1[i], y1[i]+1, list(vocabulary.keys())[i], horizontalalignment='center', size='small', color='black', weight='semibold');


In [ ]:
a= torch.rand(3, 3)   # 0.0 ~ 1.0 사이의 값을 균등분포로 출력,
print(a)
print(torch.randn(3,3))  평균이 0 이고 분산이 1인 가우시안 정규 분포로 출력